In [1]:
import sys
import os

# Add project root (one directory above "notebooks")
sys.path.append(os.path.abspath(".."))

In [5]:
import pandas as pd
from scripts.scrape_reviews import scrape_reviews_for_app

In [6]:


APPS = {
    "CBE": "com.combanketh.mobilebanking",  #https://play.google.com/store/apps/details?id=com.combanketh.mobilebanking&hl=en
    "BOA": "com.boa.boaMobileBanking",    #https://play.google.com/store/apps/details?id=com.boa.boaMobileBanking&pcampaignid=web_share
    "Dashen": "com.dashen.dashensuperapp",  #https://play.google.com/store/apps/details?id=com.dashen.dashensuperapp&pcampaignid=web_share
}



In [7]:
all_reviews = []
for bank, package in APPS.items():
    data = scrape_reviews_for_app(bank, package)
    all_reviews.extend(data)

🔹 Scraping CBE...
✅ Finished CBE (400 reviews)
🔹 Scraping BOA...
✅ Finished BOA (400 reviews)
🔹 Scraping Dashen...
✅ Finished Dashen (400 reviews)


In [8]:
df = pd.DataFrame(all_reviews)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,bank
0,3d88a334-958c-4717-9f97-c5d46359e054,samson getachew,https://play-lh.googleusercontent.com/a/ACg8oc...,It is nice app,5,0,5.2.1,2025-11-26 12:03:18,None,NaT,5.2.1,CBE
1,99d376ea-4824-4af9-a093-27360acc3a5c,Nejbadin Ali,https://play-lh.googleusercontent.com/a-/ALV-U...,best,5,0,5.2.1,2025-11-25 20:27:20,None,NaT,5.2.1,CBE
2,f1861daf-a1ed-407a-9e7c-295edbb3877d,Amman Mom,https://play-lh.googleusercontent.com/a/ACg8oc...,good app,5,0,5.2.1,2025-11-25 18:10:35,None,NaT,5.2.1,CBE
3,fd178fb7-7026-4d02-98a0-5c86c3bd56f5,Nuredin Adem,https://play-lh.googleusercontent.com/a/ACg8oc...,it suddenly asked me to enter the verification...,1,0,5.2.1,2025-11-25 17:14:22,None,NaT,5.2.1,CBE
4,571c66c6-fd18-437b-b8e5-8c443e9db2df,Abdulfeta Mifta,https://play-lh.googleusercontent.com/a/ACg8oc...,nice app...,5,0,None,2025-11-25 10:24:19,None,NaT,None,CBE


In [7]:
df.to_csv("../data/raw_bank_reviews.csv", index=False)
print("💾 Saved to data/raw_bank_reviews.csv")


💾 Saved to data/raw_bank_reviews.csv
